Ссылка на данный блокнот: https://colab.research.google.com/drive/1QJhmaAGopzt9t7oh9CV420mWCQAGD63i?usp=drive_link

Чтобы в Google Collab работал PySpark необходимо установить на виртуальное окружение пакет Java, скачать архив Spark, распаковать его и для Python установить библиотеку findspark.


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

Добавляем значения "JAVA_HOME" и "SPARK_HOME" в переменные среды с помощью метода "os.environ"   и указываем каталоги, где находятся Java и пакет Spark. Объявляем "Master" в нашем случае он локальный ("local[*]").


In [ ]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark

findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)  # Property used to format output tables better
spark


Устанавливаем и подключаем необходимые для выполнения задания библиотеки, описанные в README.md и подключаем Google Drive, чтобы сохранять сгенерированные файлы локально.

In [ ]:
!pip install country_list
!pip install countryinfo

import random
import uuid
import string
import pyspark.sql.types as T
import pyspark.sql.functions as F
import countryinfo
import country_list
import hashlib
import datetime
from google.colab import drive

drive.mount("/content/gdrive")

Создаём функцию "generate_string", которая позволит сгенерировать список необходимых данных представленных для дальнейшего заполнения таблиц(витрин) из файла "Витрины.docx". Аргументами данной функции является число строк, которые мы хотим вставить в таблицу за раз и дата в поле "timestampcolumn".

In [ ]:
countries = list(country_list.countries_for_language('en'))


def gen_number_to_str(count: int = 10):
    '''Функция, позволяющая сгенерировать последовательность из чисел.
       Возвращающийся тип данных "String"
       На вход функции подается длинна последовательности чисел.(по умолчанию
       равна 10).
    '''
    return ''.join(random.choices(string.digits, k=count))


def generate_string(count: int = 1,
                    timestampcolumn: datetime = datetime.date.today()):
    ''' Функция генерирует записи для дальнейшей таблицы
        Возвращает массив списков
        На вход первый параметр отвечает за количество строк таблицы за 1 проход,
        второй за дату которая будет проставлена в колонке "timestampcolumn"
    '''
    list_cookies = list()
    for _ in range(count):
        # генерация cookies
        INN = gen_number_to_str(12)

        _sa_cookie_a = {
            "key": "_sa_cookie_a",
            "value": f"SA1.{uuid.uuid4()}.{gen_number_to_str(10)}"
        }
        _fa_cookie_a = {"key": "_fa_cookie_a",
                        "value": f"ads2."
                                 f"{gen_number_to_str(10)}."
                                 f"{gen_number_to_str(10)}"}

        _ym_cookie_c = {"key": "_ym_cookie_c",
                        "value": {''.join(gen_number_to_str(20))}}

        _fbp = {"key": "_fbp",
                "value": f"fb.{random.choice(string.digits)}."
                         f"{gen_number_to_str(13)}."
                         f"{gen_number_to_str(9)}"}

        org_uid = {"key": "org_uid",
                   "value": f"{gen_number_to_str(7)}"}

        user_uid = {"key": "user_uid",
                    "value": f"{gen_number_to_str(7)}"}

        user_phone = {"key": "user_phone",
                      "value": f"7 9{gen_number_to_str(2)} "
                               f"{gen_number_to_str(3)} "
                               f"{gen_number_to_str(2)} "
                               f"{gen_number_to_str(2)}"}

        user_mail = {
            "key": "user_mail",
            "value": f'''{''.join(random.choices(string.ascii_letters +
                                                 string.digits, k=10))}@user.io'''}

        # генерация event_type
        event_type = random.choice(["SUBMIT", "REGISTER", "SUBMIT_MD5"])

        # генерация event_action
        event_action = random.choice(["pageview", "event", "login-check-otp"])

        # генерация data_value
        if event_type == "SUBMIT":
            data_value = hashlib.sha256(bytes(INN, encoding="utf-8")).hexdigest()
        elif event_type == "SUBMIT_MD5":
            data_value = hashlib.md5(bytes(INN, encoding="utf-8")).hexdigest()
        else:
            data_value = None

        # генерация страный, города и геопозиции
        flag = True
        while flag:
            try:
                geocountry = random.choice(countries)
                country = countryinfo.CountryInfo(geocountry[1])
                city = country.capital()
                geoaltitude = ','.join(map(str, country.latlng()))
                flag = False
            except:
                with open("Errors.txt", "a", encoding="UTF-8") as file:
                    file.writelines(
                        f'{datetime.datetime.now()} - Ошибка "NotContryInList"'
                        f'информация о стране {geocountry[1]}'
                        f'отсутвует в библиотеке "countryinfo"\n')
                continue

        # Генерация meta_platform
        meta_platform = random.choice(["WEB", "MOBAIL"])

        # Генерация user_os
        if meta_platform == "WEB":
            user_os = random.choice(["Mac", "Windows", "Ubuntu"])
        else:
            user_os = random.choice(["IOS", "Android", "HarmonyOS", "BlackBerryOS"])
        # Генерация systemlanguage
        systemlanguage = random.choice(["RU", "ENG"])

        # Генерация screensize
        screensize = "1920x1080"

        # Добавление спосков в единый список
        list_cookies.append([INN,
                             [_sa_cookie_a, _fa_cookie_a, _ym_cookie_c, _fbp,
                              org_uid, user_uid, user_phone, user_mail],
                             event_type,
                             event_action,
                             data_value,
                             geocountry[1],
                             city,
                             user_os,
                             systemlanguage,
                             geoaltitude,
                             meta_platform,
                             screensize,
                             timestampcolumn
                             ])
    return list_cookies

Объявление схемы данных таблицы

In [ ]:
# схема данных
schema = T.StructType([T.StructField("INN", T.StringType(), True),
                       T.StructField("raw_cookie", T.ArrayType(T.MapType(T.StringType(), T.StringType()))),
                       T.StructField("event_type", T.StringType(), True),
                       T.StructField("event_action", T.StringType(), True),
                       T.StructField("data_value", T.StringType(), True),
                       T.StructField("geocountry", T.StringType(), True),
                       T.StructField("city", T.StringType(), True),
                       T.StructField("user_os", T.StringType(), True),
                       T.StructField("systemlanguage", T.StringType(), True),
                       T.StructField("geoaltitude", T.StringType(), True),
                       T.StructField("meta_platform", T.StringType(), True),
                       T.StructField("screensize", T.StringType(), True),
                       T.StructField("timestampcolumn", T.DateType(), True)
                       ])

Цикл, генерируемый записи таблицы и сохраняемый их в файлы формата JSON, в
каталоги по дням недели. Все каталоги создаются автоматически. Данный цикл эмулирует постоянное добавление файлов. Необходимо запустить в фоновом режиме, пока идет работа с файлом "Create_data_marts.ipynb". Это требуется, чтобы смоделировать приближенную картину к реальности при работе с витриной "B" и витриной "D", так как они одинаковые, но количество строк в них может быть разным.

Чтобы не допустить переполнение диска, цикл отработает 10 раз после чего
выключится самостоятельно.



In [ ]:
# Цикл генерирует каталоги с файлами в формате JSON согласно партиционирования
first_date = datetime.date.today()
folder = "/content/gdrive/MyDrive/data/json/"
if not os.path.isdir(folder):
    os.makedirs(folder)
i = 0
while True:
    for _ in range(random.randrange(5, 10)):
        df = spark.createDataFrame(generate_string(30, first_date), schema=schema)
        df.coalesce(1).write.mode("append") \
            .json(f"{folder}{first_date.strftime('%Y_%m_%d')}")

    first_date = first_date + datetime.timedelta(1)
    i = i + 1
    if i == 10:  # ограничитель на 10 сгенерированных частей
        break
